In [1]:
from xml.etree.ElementTree import Element, SubElement, Comment, tostring
from xml.etree import ElementTree
from xml.dom import minidom
import pandas as pd
from PIL import Image
import shutil
def prettify(elem):
    """Return a pretty-printed XML string for the Element.
    """
    rough_string = ElementTree.tostring(elem, 'utf-8')
    reparsed = minidom.parseString(rough_string)
    return reparsed.toprettyxml(indent="  ")

In [2]:
annotation_data = pd.read_csv("fileList.csv")

In [3]:
annotation_data.count()

File          2451
fileName      2451
ObjectName    2451
xmin          2451
ymin          2451
xmax          2451
ymax          2451
dtype: int64

In [7]:
for i in range(len(annotation_data)):
    nameList = annotation_data.fileName.iloc[i].split("_")
    if len(nameList) >2:
        #newName =  annotation_data.ObjectName.iloc[i] + "_" + annotation_data.fileName.iloc[i]
        newName = nameList[0].replace("-","_") + "_" + nameList[2]
        shutil.copy2('all_images/'+annotation_data.fileName.iloc[i], 'images/'+newName)
        annotation_data.loc[i,'fileName'] = newName
    else:
        shutil.copy2('all_images/'+annotation_data.fileName.iloc[i], 'images/'+annotation_data.fileName.iloc[i])
    
#annotation_data.fileName = annotation_data.ObjectName + "_" + annotation_data.fileName

C:\Users\Admin\Anaconda2\lib\site-packages\pandas\core\indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


## Create Annotation XML files for each image with bounding boxes

In [8]:
for fName in annotation_data.fileName.unique():
    image_dataset = annotation_data[annotation_data['fileName']==fName]
    #print image_dataset
    im=Image.open('images/'+fName)
     # (width,height) tuple

    top = Element('annotation')
    folder = SubElement(top, 'folder')
    folder.text = 'BelgaLogos'
    filename = SubElement(top, 'filename')
    filename.text = fName

    source = SubElement(top, 'source')
    database = SubElement(source, 'database')
    database.text = "BelgaLogos database"

    annotation = SubElement(source,'annotation')
    annotation.text =  "BelgaLogos database"

    image = SubElement(source,'image')
    image.text = 'BelgaLogos'

    size = SubElement(top,'size')
    width = SubElement(size,'width')
    width.text = str(im.size[0])
    height = SubElement(size,'height')
    height.text = str(im.size[1])
    depth = SubElement(size,'depth')
    depth.text = '3'

    segmented =SubElement(top,'segmented')
    segmented.text = '1'

    for i in range(len(image_dataset)):
        y = image_dataset.iloc[i]

        object = SubElement(top,'object')

        name =SubElement(object,'name')
        name.text = y.ObjectName
        pose = SubElement(object,'pose')
        pose.text = 'pose'
        truncated = SubElement(object,'truncated')
        truncated.text= '0'
        difficult =SubElement(object,'difficult')
        difficult.text = '0'

        bndbox = SubElement(object,'bndbox')
        xmin = SubElement(bndbox,'xmin')
        xmin.text = str(y.xmin)
        ymin = SubElement(bndbox,'ymin')
        ymin.text = str(y.ymin)
        xmax = SubElement(bndbox,'xmax')
        xmax.text = str(y.xmax)
        ymax = SubElement(bndbox,'ymax')
        ymax.text = str(y.ymax)


    #print prettify(top)

    f = open('annotations/xmls/'+fName.split(".")[0]+'.xml', 'w')
    f.write(tostring(top))  # python will convert \n to os.linesep
    f.close()
    
print "Done"

Done


In [20]:
import glob
import pandas as pd
files = glob.glob("E:\\Object Detection\\Coke\\images\\*")
annotation_data = pd.DataFrame(files,columns=['fileName'])
annotation_data.fileName = annotation_data.fileName.apply(lambda x: x.split("\\")[-1].split(".")[0])
annotation_data['ObjectName'] = 'CocaCola'


## Create List Train and Val.txt files

In [23]:
objects = list(annotation_data.ObjectName.unique())
annotation_data_new = annotation_data.copy()
annotation_data_new.fileName = annotation_data.fileName.apply(lambda x: x.split(".")[0])
annotation_data_new.ObjectName = annotation_data.ObjectName.apply(lambda x: objects.index(x)+1)
#annotation_data_new= annotation_data_new.drop(['xmin','ymin','xmax','ymax'], axis=1)
#annotation_data_new= annotation_data_new.drop(['File','xmin','ymin','xmax','ymax'], axis=1)

annotation_data_new.to_csv('annotations/list.txt',sep=' ',header=None,index=None)

train = pd.DataFrame(columns=['fileName','ObjectName'])
test = pd.DataFrame(columns=['fileName','ObjectName'])
for i in range(len(objects)+1):
    train = train.append(annotation_data_new[annotation_data_new['ObjectName']==i].iloc[50:,])
    test = test.append(annotation_data_new[annotation_data_new['ObjectName']==i].iloc[:50,])
    
train.ObjectName = train.ObjectName.astype(int)
test.ObjectName = test.ObjectName.astype(int)

train.to_csv('annotations/trainval.txt',sep=' ',header=None,index=None)
test.to_csv('annotations/test.txt',sep=' ',header=None,index=None)